## Get cumulative number of peaks discovered with increasing number of tissues
(greedy algorithm)
a mRNA TSS  
b. unstable TSS  
c. lncRNA  
d. unstable (eRNA)  
e. miRNA  
f. other  
g. ATAC peaks  

In [1]:
import yaml
import os
import pandas as pd
## Parameters specific to where your folders are and your data
parameter_file = '../parameters/params.yaml'

samples = pd.read_csv('../parameters/samples.tsv',sep='\t', index_col=0)
import sys
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
#anno_gff=doc['annotation']
os.chdir(doc["results"])
#mRNA_peak_file = doc["mRNA_peak_file"]
#tss_annotation = doc['tss_annotation']

import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from os.path import join
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *
import helper
import create_output
print('Number of tissues: ',len(tissues))

homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # Remove the CWD from sys.path while we load stuff.


Number of tissues:  13
/data2/resources/software/homer/v4.11_10242019/bin//homer


In [2]:
samples

,Experiment,Full,Name,Tissue
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...,BMDM1hKLA_ATAC1,BMDM1hKLA
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...,BMDM1hKLA_ATAC2,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,GRO,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...,BMDM1hKLA_GRO1,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,GROCap,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...,BMDM1hKLA_GROCap1,BMDM1hKLA
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,ATAC,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...,BMDMwt_ATAC1,BMDMwt
...,...,...,...,...
NL5_Liver_S5_L006_R1_001.fastq.gz,RNASeq,NL5_Liver_S5_L006_R1_001.fastq.gz,Liver_RNASeq1,Liver
NL6_Heart_S6_L006_R1_001.fastq.gz,RNASeq,NL6_Heart_S6_L006_R1_001.fastq.gz,Heart_RNASeq1,Heart
NL7_Brain2_S7_L006_R1_001.fastq.gz,RNASeq,NL7_Brain2_S7_L006_R1_001.fastq.gz,Brain_RNASeq2,Brain
NL8_Muscle_S8_L006_R1_001.fastq.gz,RNASeq,NL8_Muscle_S8_L006_R1_001.fastq.gz,Muscle_RNASeq1,Muscle


In [3]:
anno_gtf_f = doc['annotation_gtf']
print(anno_gtf_f)

/data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.gtf


In [4]:
anno_f = join('merged','samples.merge.anno')
peaks_expr_f = join('merged','tissues.merge.peaksexpression')


## Create a tissue_experiment dataframe of each tissue and the file needed. Some will be Na
## Then for TSS, create the cumsum function To get the order of tissues.
## After that, loop through tissue_experiment df and calculate the number of elements added 
## Plot across experiments

## Load peak annotations

In [5]:
peaks_with_tss_distances_size1 = pd.read_csv('merged/peaks_with_tss_distances_size1.noCDS.tsv',sep='\t')
peaks_with_tss_distances_size1

,ID,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene
0,0,NW_020822452.1,16590385,16590386,+,16.227333,promoter-TSS (ID=exon-XM_027399668.1-1;Parent=...,XM_027399668.1,-462.5,False,Clic4
1,1,NW_020822597.1,23595396,23595397,+,5.740000,Intergenic,XM_027420351.1,-23849.5,True,Baz2b
2,2,NW_020822680.1,728718,728719,+,7.579579,promoter-TSS (ID=exon-XM_027432478.1-3;Parent=...,XM_027432478.1,746.0,False,Irak1
3,3,NW_020822602.1,10883205,10883206,+,3.992000,Intergenic,XM_027423054.1,9079.5,True,Gbgt1
4,4,NW_020822601.1,37427907,37427908,+,129.031738,promoter-TSS (ID=exon-XM_027421990.1-1;Parent=...,XM_027421990.1,-10.5,True,Shf
...,...,...,...,...,...,...,...,...,...,...,...
515559,540122,NW_020822538.1,6114468,6114469,+,7.122000,promoter-TSS (ID=exon-XM_027415967.1-1;Parent=...,XM_027415967.1,-58.5,False,Selplg
515560,540123,NW_020822487.1,8747242,8747243,+,7.508000,Intergenic,XM_027405626.1,-7856.0,True,Herpud1
515561,540124,NW_020822407.1,2824603,2824604,+,3.612167,Intergenic,XM_027390727.1,-132495.0,True,Klf3
515562,540125,NW_020822545.1,3178880,3178881,-,2.400286,TTS (ID=exon-XM_027416357.1-1;Parent=rna-XM_02...,XM_027416357.1,1382.5,True,Ccdc85c


In [6]:
ncrnas = peaks_with_tss_distances_size1[peaks_with_tss_distances_size1['Annotation'].fillna('').str.contains("ncRNA")]
print(f"Number of ncrnas: {ncrnas.shape[0]}")

for curr_anno in ncrnas["Annotation"].values:
    print(curr_anno)

Number of ncrnas: 12184
promoter-TSS (ID=exon-XR_003485658.1-3;Parent=rna-XR_003485658.1;Dbxref=GeneID:103159771,Genbank:XR_003485658.1;gbkey=ncRNA;gene=LOC103159771;product=uncharacterized LOC103159771%2C transcript variant X1;transcript_id=XR_003485658.1)
promoter-TSS (ID=exon-XR_003485972.1-2;Parent=rna-XR_003485972.1;Dbxref=GeneID:113836013,Genbank:XR_003485972.1;gbkey=ncRNA;gene=LOC113836013;product=uncharacterized LOC113836013;transcript_id=XR_003485972.1)
promoter-TSS (ID=exon-XR_003486078.1-1;Parent=rna-XR_003486078.1;Dbxref=GeneID:113836093,Genbank:XR_003486078.1;gbkey=ncRNA;gene=LOC113836093;product=uncharacterized LOC113836093%2C transcript variant X2;transcript_id=XR_003486078.1)
promoter-TSS (ID=exon-XR_003485260.1-1;Parent=rna-XR_003485260.1;Dbxref=GeneID:113835601,Genbank:XR_003485260.1;gbkey=ncRNA;gene=LOC113835601;product=uncharacterized LOC113835601;transcript_id=XR_003485260.1)
TTS (ID=exon-XR_003485314.1-3;Parent=rna-XR_003485314.1;Dbxref=GeneID:113835635,Genbank:XR

TTS (ID=exon-XR_003481110.1-1;Parent=rna-XR_003481110.1;Dbxref=GeneID:113833432,Genbank:XR_003481110.1;gbkey=ncRNA;gene=LOC113833432;product=uncharacterized LOC113833432;transcript_id=XR_003481110.1)
TTS (ID=exon-XR_003481105.1-1;Parent=rna-XR_003481105.1;Dbxref=GeneID:113833431,Genbank:XR_003481105.1;gbkey=ncRNA;gene=LOC113833431;product=uncharacterized LOC113833431;transcript_id=XR_003481105.1)
promoter-TSS (ID=exon-XR_003487086.1-4;Parent=rna-XR_003487086.1;Dbxref=GeneID:103164258,Genbank:XR_003487086.1;gbkey=ncRNA;gene=LOC103164258;product=uncharacterized LOC103164258%2C transcript variant X1;transcript_id=XR_003487086.1)
TTS (ID=exon-XR_003485955.1-1;Parent=rna-XR_003485955.1;Dbxref=GeneID:103162028,Genbank:XR_003485955.1;gbkey=ncRNA;gene=LOC103162028;product=uncharacterized LOC103162028%2C transcript variant X1;transcript_id=XR_003485955.1)
TTS (ID=exon-XR_003487148.1-1;Parent=rna-XR_003487148.1;Dbxref=GeneID:113836700,Genbank:XR_003487148.1;gbkey=ncRNA;gene=LOC113836700;product=

TTS (ID=exon-XR_003487624.1-1;Parent=rna-XR_003487624.1;Dbxref=GeneID:113831510,Genbank:XR_003487624.1;gbkey=ncRNA;gene=LOC113831510;product=uncharacterized LOC113831510%2C transcript variant X1;transcript_id=XR_003487624.1)
TTS (ID=exon-XR_003487644.1-2;Parent=rna-XR_003487644.1;Dbxref=GeneID:113837043,Genbank:XR_003487644.1;gbkey=ncRNA;gene=LOC113837043;product=uncharacterized LOC113837043%2C transcript variant X2;transcript_id=XR_003487644.1)
promoter-TSS (ID=exon-XR_003487630.1-4;Parent=rna-XR_003487630.1;Dbxref=GeneID:113837037,Genbank:XR_003487630.1;gbkey=ncRNA;gene=LOC113837037;product=uncharacterized LOC113837037%2C transcript variant X1;transcript_id=XR_003487630.1)
TTS (ID=exon-XR_003487668.1-2;Parent=rna-XR_003487668.1;Dbxref=GeneID:113837049,Genbank:XR_003487668.1;gbkey=ncRNA;gene=LOC113837049;product=uncharacterized LOC113837049%2C transcript variant X3;transcript_id=XR_003487668.1)
promoter-TSS (ID=exon-XR_003487644.1-2;Parent=rna-XR_003487644.1;Dbxref=GeneID:113837043,Ge

TTS (ID=exon-XR_003487413.1-1;Parent=rna-XR_003487413.1;Dbxref=GeneID:103164366,Genbank:XR_003487413.1;gbkey=ncRNA;gene=LOC103164366;product=uncharacterized LOC103164366%2C transcript variant X1;transcript_id=XR_003487413.1)
exon (ID=exon-XR_003487781.1-2;Parent=rna-XR_003487781.1;Dbxref=GeneID:113837114,Genbank:XR_003487781.1;gbkey=ncRNA;gene=LOC113837114;product=uncharacterized LOC113837114;transcript_id=XR_003487781.1, exon 1 of 1)
promoter-TSS (ID=exon-XR_003485755.1-2;Parent=rna-XR_003485755.1;Dbxref=GeneID:107978306,Genbank:XR_003485755.1;gbkey=ncRNA;gene=LOC107978306;product=uncharacterized LOC107978306;transcript_id=XR_003485755.1)
promoter-TSS (ID=exon-XR_003481303.1-1;Parent=rna-XR_003481303.1;Dbxref=GeneID:103162281,Genbank:XR_003481303.1;gbkey=ncRNA;gene=LOC103162281;product=uncharacterized LOC103162281%2C transcript variant X2;transcript_id=XR_003481303.1)
promoter-TSS (ID=exon-XR_003485316.1-3;Parent=rna-XR_003485316.1;Dbxref=GeneID:107978623,Genbank:XR_003485316.1;gbkey=

promoter-TSS (ID=exon-XR_003482758.1-2;Parent=rna-XR_003482758.1;Dbxref=GeneID:113834313,Genbank:XR_003482758.1;gbkey=ncRNA;gene=LOC113834313;product=uncharacterized LOC113834313;transcript_id=XR_003482758.1)
TTS (ID=exon-XR_003480331.1-1;Parent=rna-XR_003480331.1;Dbxref=GeneID:113832857,Genbank:XR_003480331.1;gbkey=ncRNA;gene=LOC113832857;product=uncharacterized LOC113832857;transcript_id=XR_003480331.1)
promoter-TSS (ID=exon-XR_003484425.1-1;Parent=rna-XR_003484425.1;Dbxref=GeneID:103164539,Genbank:XR_003484425.1;gbkey=ncRNA;gene=LOC103164539;product=uncharacterized LOC103164539%2C transcript variant X2;transcript_id=XR_003484425.1)
promoter-TSS (ID=exon-Mir132-1;Parent=rna-Mir132;Dbxref=GeneID:102466525,miRBase:MIMAT0023759,miRBase:MI0020396;gbkey=ncRNA;gene=Mir132;product=cgr-miR-132-3p)
promoter-TSS (ID=exon-XR_003486956.1-1;Parent=rna-XR_003486956.1;Dbxref=GeneID:103160777,Genbank:XR_003486956.1;gbkey=ncRNA;gene=LOC103160777;product=uncharacterized LOC103160777%2C transcript vari

promoter-TSS (ID=exon-XR_003480734.1-2;Parent=rna-XR_003480734.1;Dbxref=GeneID:103159465,Genbank:XR_003480734.1;gbkey=ncRNA;gene=LOC103159465;product=uncharacterized LOC103159465%2C transcript variant X1;transcript_id=XR_003480734.1)
exon (ID=exon-XR_003485812.1-4;Parent=rna-XR_003485812.1;Dbxref=GeneID:103160324,Genbank:XR_003485812.1;gbkey=ncRNA;gene=LOC103160324;product=uncharacterized LOC103160324%2C transcript variant X8;transcript_id=XR_003485812.1, exon 1 of 1)
promoter-TSS (ID=exon-XR_003484344.1-1;Parent=rna-XR_003484344.1;Dbxref=GeneID:113835103,Genbank:XR_003484344.1;gbkey=ncRNA;gene=LOC113835103;product=uncharacterized LOC113835103%2C transcript variant X1;transcript_id=XR_003484344.1)
promoter-TSS (ID=exon-XR_003487995.1-3;Parent=rna-XR_003487995.1;Dbxref=GeneID:107977965,Genbank:XR_003487995.1;gbkey=ncRNA;gene=LOC107977965;product=uncharacterized LOC107977965%2C transcript variant X4;transcript_id=XR_003487995.1)
promoter-TSS (ID=exon-XR_003484344.1-2;Parent=rna-XR_003484

In [7]:
peaks_with_tss_distances = pd.read_csv('merged/peaks_with_tss_distances.tsv',sep='\t')
peaks_with_tss_distances

,ID,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene
0,0,NW_020822452.1,16590187,16590584,+,16.227333,promoter-TSS (ID=exon-XM_027399668.1-1;Parent=...,XM_027399668.1,-462.5,False,Clic4
1,1,NW_020822597.1,23595304,23595489,+,5.740000,Intergenic,XM_027420351.1,-23849.5,True,Baz2b
2,2,NW_020822680.1,728607,728829,+,7.579579,promoter-TSS (ID=exon-XM_027432478.1-3;Parent=...,XM_027432478.1,746.0,False,Irak1
3,3,NW_020822602.1,10883074,10883337,+,3.992000,Intergenic,XM_027423054.1,9079.5,True,Gbgt1
4,4,NW_020822601.1,37427816,37427999,+,129.031738,promoter-TSS (ID=exon-XM_027421990.1-1;Parent=...,XM_027421990.1,-10.5,True,Shf
...,...,...,...,...,...,...,...,...,...,...,...
540122,540122,NW_020822538.1,6114376,6114561,+,7.122000,promoter-TSS (ID=exon-XM_027415967.1-1;Parent=...,XM_027415967.1,-58.5,False,Selplg
540123,540123,NW_020822487.1,8747140,8747344,+,7.508000,Intergenic,XM_027405626.1,-7856.0,True,Herpud1
540124,540124,NW_020822407.1,2824447,2824759,+,3.612167,Intergenic,XM_027390727.1,-132495.0,True,Klf3
540125,540125,NW_020822545.1,3178759,3179002,-,2.400286,TTS (ID=exon-XM_027416357.1-1;Parent=rna-XM_02...,XM_027416357.1,1382.5,True,Ccdc85c
